# Segmenting and Clustering Neighborhoods in Toronto

by _Vicente Carvalho_

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load Prepared Data

In [4]:
df = pd.read_csv('toronto_postcode_latlong.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722079,-79.437507
4,M7A,Downtown Toronto,Queen's Park,43.660609,-79.390573


### Use geopy library to get the latitude and longitude values of New York City.

In [5]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Create a map of Toronto with Neighborhoods

In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'ZZUKG2R2MDGSCP30TS4XINPAP4Y4PMBLU1DO1TNGFFPCNTZR' # your Foursquare ID
CLIENT_SECRET = 'KTAN34ECK3NKIB2LVQD2REJ5IEF4CG1YZOOEA5A3RIXSMMIT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZZUKG2R2MDGSCP30TS4XINPAP4Y4PMBLU1DO1TNGFFPCNTZR
CLIENT_SECRET:KTAN34ECK3NKIB2LVQD2REJ5IEF4CG1YZOOEA5A3RIXSMMIT


### Define function to get category

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Define function to get category for all Neighborhoods

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Call Function to get all information for all Neighborhoods

In [11]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names= df['Neighborhood'],
                                   latitudes= df['Latitude'],
                                   longitudes= df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Woodbine Gardens, Parkview Hill
Ryerson, Garden District
Glencairn
Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park
Highland Creek, Rouge Hill, Port Union
Flemingdon Park, Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Downsview North, Wilson Heights
Thorncliffe Park
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
East Birchmount Park, Ionview, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

#### Let's check the size of the resulting dataframe

In [12]:
print(toronto_venues.shape)
toronto_venues.head()

(3140, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
3,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
4,Parkwoods,43.7588,-79.320197,TD Canada Trust,43.757569,-79.314976,Bank


In [13]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,10,10,10,10,10,10
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",5,5,5,5,5,5
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",60,60,60,60,60,60
"Alderwood, Long Branch",12,12,12,12,12,12
"Bathurst Manor, Downsview North, Wilson Heights",5,5,5,5,5,5
Bayview Village,10,10,10,10,10,10
"Bedford Park, Lawrence Manor East",5,5,5,5,5,5
Berczy Park,100,100,100,100,100,100


In [14]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 284 uniques categories.


### Analyze Each Neighborhood

In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [16]:
toronto_onehot.shape

(3140, 284)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport Service,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,River,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Tunnel,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [18]:
toronto_grouped.shape

(102, 284)

#### Let's print each neighborhood along with the top 5 most common venues

In [19]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                venue  freq
0         Coffee Shop  0.09
1               Hotel  0.06
2          Restaurant  0.06
3                Café  0.05
4  Italian Restaurant  0.04


----Agincourt----
                   venue  freq
0     Chinese Restaurant   0.3
1      Korean Restaurant   0.1
2            Coffee Shop   0.1
3   Cantonese Restaurant   0.1
4  Vietnamese Restaurant   0.1


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0       Shopping Mall   0.2
1  Chinese Restaurant   0.2
2            Bus Stop   0.2
3         Coffee Shop   0.2
4  Athletics & Sports   0.2


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
            venue  freq
0     Coffee Shop  0.13
1  Farmers Market  0.05
2            Café  0.05
3            Park  0.05
4  Cosmetics Shop  0.03


----Alderwood, Long Branch----
                venue  freq
0  Light Rail Station  0.17
1  Se

                 venue  freq
0                  Pub  0.17
1        Grocery Store  0.17
2  Japanese Restaurant  0.17
3         Tennis Court  0.17
4           Playground  0.17


----Guildwood, Morningside, West Hill----
            venue  freq
0     Coffee Shop  0.15
1            Park  0.15
2  Sandwich Place  0.15
3        Pharmacy  0.08
4      Beer Store  0.08


----Harbord, University of Toronto----
                 venue  freq
0                 Café  0.16
1  Japanese Restaurant  0.06
2   Italian Restaurant  0.06
3                 Park  0.06
4            Bookstore  0.06


----Harbourfront----
                venue  freq
0         Coffee Shop  0.15
1                Café  0.05
2          Restaurant  0.04
3               Hotel  0.04
4  Italian Restaurant  0.03


----Harbourfront East, Toronto Islands, Union Station----
                 venue  freq
0                 Pier  0.14
1          Beer Garden  0.14
2  American Restaurant  0.14
3                Beach  0.14
4                 Farm  0.1

4            Thai Restaurant   0.2


----Westmount----
                     venue  freq
0                     Park   0.5
1              Pizza Place   0.5
2  New American Restaurant   0.0
3     Outdoor Supply Store   0.0
4             Optical Shop   0.0


----Weston----
           venue  freq
0    Coffee Shop  0.12
1  Train Station  0.12
2          Diner  0.06
3     Laundromat  0.06
4   Soccer Field  0.06


----Willowdale South----
                  venue  freq
0           Coffee Shop  0.14
1   Japanese Restaurant  0.08
2         Grocery Store  0.06
3  Fast Food Restaurant  0.06
4            Restaurant  0.04


----Willowdale West----
                  venue  freq
0           Coffee Shop  0.14
1   Japanese Restaurant  0.08
2         Grocery Store  0.06
3  Fast Food Restaurant  0.06
4            Restaurant  0.04


----Woburn----
                  venue  freq
0  Fast Food Restaurant  0.18
1           Coffee Shop  0.09
2        Discount Store  0.05
3                  Bank  0.05
4           

#### Let's put that into a *pandas* dataframe

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Restaurant,Hotel,Café,Gastropub,Japanese Restaurant,Italian Restaurant,Gym,American Restaurant,Seafood Restaurant
1,Agincourt,Chinese Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Korean Restaurant,Train Station,Asian Restaurant,Coffee Shop,Vietnamese Restaurant,Flea Market,Fish Market
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Bus Stop,Shopping Mall,Chinese Restaurant,Coffee Shop,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Festival,Filipino Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Coffee Shop,Café,Park,Farmers Market,Pet Store,Cheese Shop,Bakery,Cosmetics Shop,Seafood Restaurant,Theater
4,"Alderwood, Long Branch",Light Rail Station,Burrito Place,Dog Run,Liquor Store,Trail,Park,Seafood Restaurant,Pharmacy,Café,South American Restaurant


### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [41]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 3, 0, 0, 0, 0], dtype=int32)

In [42]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,0,Gas Station,Convenience Store,Laundry Service,Liquor Store,Shopping Mall,Bank,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Pharmacy
1,M4A,North York,Victoria Village,43.732658,-79.311189,0,Middle Eastern Restaurant,Thai Restaurant,Asian Restaurant,Spa,Bus Line,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop
2,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Music Venue,Sports Bar,Steakhouse,Fried Chicken Joint,Sushi Restaurant
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.722079,-79.437507,0,Pharmacy,Park,Bank,Electronics Store,Kids Store,Financial or Legal Service,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant
4,M7A,Downtown Toronto,Queen's Park,43.660609,-79.390573,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Park,Indian Restaurant,Bookstore,Sushi Restaurant,Bubble Tea Shop,Fried Chicken Joint


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster 1

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Gas Station,Convenience Store,Laundry Service,Liquor Store,Shopping Mall,Bank,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Pharmacy
1,North York,0,Middle Eastern Restaurant,Thai Restaurant,Asian Restaurant,Spa,Bus Line,French Restaurant,Fountain,Food Truck,Food Court,Food & Drink Shop
2,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Music Venue,Sports Bar,Steakhouse,Fried Chicken Joint,Sushi Restaurant
3,North York,0,Pharmacy,Park,Bank,Electronics Store,Kids Store,Financial or Legal Service,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant
4,Downtown Toronto,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Park,Indian Restaurant,Bookstore,Sushi Restaurant,Bubble Tea Shop,Fried Chicken Joint
5,Queen's Park,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Park,Indian Restaurant,Bookstore,Sushi Restaurant,Bubble Tea Shop,Fried Chicken Joint
6,Scarborough,0,Pizza Place,Pharmacy,Sandwich Place,Park,Grocery Store,Fried Chicken Joint,Gym / Fitness Center,Bubble Tea Shop,Convenience Store,Skating Rink
7,North York,0,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Women's Store,Bakery,Food Court,Sandwich Place,Shopping Mall,Convenience Store
8,East York,0,Construction & Landscaping,Women's Store,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish Market
9,Downtown Toronto,0,Coffee Shop,Clothing Store,Cosmetics Shop,Restaurant,Hotel,Middle Eastern Restaurant,Fast Food Restaurant,Bookstore,Sporting Goods Shop,Burger Joint


#### Cluster 2

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Etobicoke,1,Park,Women's Store,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
33,North York,1,Park,Tennis Court,Women's Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
70,Etobicoke,1,Park,Pizza Place,Women's Store,Financial or Legal Service,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop
86,Central Toronto,1,Park,Bank,Playground,Women's Store,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service
91,Downtown Toronto,1,Park,Playground,Bike Trail,Women's Store,Fish & Chips Shop,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service
95,Scarborough,1,Park,Fast Food Restaurant,Women's Store,Exhibit,Farm,Farmers Market,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop


#### Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York,2,Performing Arts Venue,Women's Store,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop
100,East Toronto,2,Performing Arts Venue,Women's Store,Exhibit,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop


#### Cluster 4

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,3,Gym Pool,French Restaurant,Coffee Shop,Gas Station,Gym / Fitness Center,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
40,North York,3,Gym Pool,French Restaurant,Coffee Shop,Gas Station,Gym / Fitness Center,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
46,North York,3,Gym Pool,French Restaurant,Coffee Shop,Gas Station,Gym / Fitness Center,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
53,North York,3,Gym Pool,French Restaurant,Coffee Shop,Gas Station,Gym / Fitness Center,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
60,North York,3,Gym Pool,French Restaurant,Coffee Shop,Gas Station,Gym / Fitness Center,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


#### Cluster 5

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,4,Train Station,Women's Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market
